In [9]:
import tensorflow as tf
import numpy as np

#mnist
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("./data/",one_hot=True)

Extracting ./data/train-images-idx3-ubyte.gz
Extracting ./data/train-labels-idx1-ubyte.gz
Extracting ./data/t10k-images-idx3-ubyte.gz
Extracting ./data/t10k-labels-idx1-ubyte.gz


In [10]:
n_input = 28 # MNIST data 输入 (img shape: 28*28)
n_steps = 28 # timesteps
n_hidden = 128 # hidden layer num of features
n_classes = 10  # MNIST 列别 (0-9 ，一共10类)
batch_size = 128

In [11]:
tf.reset_default_graph()
#
x = tf.placeholder("float",[None,n_steps,n_input])
y = tf.placeholder("float",[None,n_classes])

In [12]:
stacked_rnn = []
for i in range(3):
    stacked_rnn.append(tf.contrib.rnn.LSTMCell(n_hidden))
mcell = tf.contrib.rnn.MultiRNNCell(stacked_rnn)

x1 = tf.unstack(x,n_steps,1)
outputs,states = tf.contrib.rnn.static_rnn(mcell,x1,dtype=tf.float32)
pred = tf.contrib.layers.fully_connected(outputs[-1],n_classes,activation_fn=None)

In [15]:
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(logits=pred,labels=y))
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(cost)

correct_pred = tf.equal(tf.argmax(pred,1),tf.argmax(y,1))
accuracy = tf.reduce_mean(tf.cast(correct_pred,tf.float32))

In [13]:
learning_rate = 0.001
training_iters = 100000
batch_size = 128
display_step = 10

In [17]:
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    step = 1
    while step * batch_size < training_iters:
        batch_x,batch_y = mnist.train.next_batch(batch_size)
        batch_x = batch_x.reshape((batch_size,n_steps,n_input))
        sess.run(optimizer,feed_dict={x: batch_x, y: batch_y})
        if step % display_step == 0:
            acc = sess.run(accuracy,feed_dict={x: batch_x, y: batch_y})
            loss = sess.run(cost,feed_dict={x: batch_x, y: batch_y})
            print ("Iter " + str(step*batch_size) + ", Minibatch Loss= " + \
                  "{:.6f}".format(loss) + ", Training Accuracy= " + \
                  "{:.5f}".format(acc))
        step += 1
    print("Finished")
    # 计算准确率 for 128 mnist test images
    test_len = 100
    test_data = mnist.test.images[:test_len].reshape((-1, n_steps, n_input))
    test_label = mnist.test.labels[:test_len]
    print ("Testing Accuracy:", \
        sess.run(accuracy, feed_dict={x: test_data, y: test_label}))


Iter 1280, Minibatch Loss= 1.818905, Training Accuracy= 0.40625
Iter 2560, Minibatch Loss= 1.360624, Training Accuracy= 0.54688
Iter 3840, Minibatch Loss= 1.076709, Training Accuracy= 0.61719
Iter 5120, Minibatch Loss= 1.163469, Training Accuracy= 0.59375
Iter 6400, Minibatch Loss= 1.068005, Training Accuracy= 0.61719
Iter 7680, Minibatch Loss= 0.820572, Training Accuracy= 0.71094
Iter 8960, Minibatch Loss= 0.603731, Training Accuracy= 0.82031
Iter 10240, Minibatch Loss= 0.533566, Training Accuracy= 0.80469
Iter 11520, Minibatch Loss= 0.480227, Training Accuracy= 0.89062
Iter 12800, Minibatch Loss= 0.536799, Training Accuracy= 0.78125
Iter 14080, Minibatch Loss= 0.413129, Training Accuracy= 0.89062
Iter 15360, Minibatch Loss= 0.422499, Training Accuracy= 0.82812
Iter 16640, Minibatch Loss= 0.684298, Training Accuracy= 0.76562
Iter 17920, Minibatch Loss= 0.225341, Training Accuracy= 0.92188
Iter 19200, Minibatch Loss= 0.209114, Training Accuracy= 0.96875
Iter 20480, Minibatch Loss= 0.35